# Face Analysis and Fortune Reading

This notebook demonstrates how to analyze facial features and generate fortune readings based on those features. It uses a two-step process:

1. **Face Analysis**: Using Amazon Bedrock's Nova model to analyze facial features.
2. **Fortune Generation**: Creating a fortune reading based on the analyzed facial features.

## Installing Required Libraries

In [ ]:
# Install required libraries
!pip install boto3 pillow matplotlib

## Importing Required Libraries

In [ ]:
import boto3
import json
import os
import pprint
from PIL import Image
import matplotlib.pyplot as plt
from botocore.exceptions import ClientError
import logging
import base64
from io import BytesIO
from IPython.display import display, HTML, Markdown

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Verifying AWS Credentials

In [ ]:
# Check if AWS credentials are properly configured
try:
    boto3.client('sts').get_caller_identity()
    print("✅ AWS credentials are properly configured.")
except Exception as e:
    print("❌ There's an issue with AWS credential configuration.")
    print(f"Error message: {e}")
    print("Use 'aws configure' command with AWS CLI to set up credentials.")

## Image Loading and Display Function

In [ ]:
def load_and_display_image(image_path):
    """Function to load and display an image"""
    img = Image.open(image_path)
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Face Image')
    plt.show()
    return img

## Setting Image Path and Displaying Image

In [ ]:
# Set sample image path
image_path = "sample/sample_face.png"

# Load and display image
img = load_and_display_image(image_path)

## Step 1: Face Analysis Function Implementation (Using Nova Model)

In [ ]:
def analyze_face(image_path):
    """
    Function to analyze facial features from an image using Nova model
    """
    # Set model ID - Nova Lite model
    model_id = "us.amazon.nova-lite-v1:0"
    
    # Encode image as base64
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    
    # Create Bedrock client
    bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')
    
    # Create prompt
    system_prompt = "You are an AI specialized in facial analysis. Your role is to analyze facial features and extract key keywords. Do not include any negative content about the face in your output."
    
    user_prompt = """
Please describe the following facial elements of the person in this image in detail within <description> tags:

- Face shape (round, oval, angular, etc.)
- Eye characteristics (size, shape, double eyelids, etc.)
- Nose characteristics (length, shape, bridge height, etc.)
- Lip characteristics (thickness, shape, distinctive features, etc.)
- Skin characteristics (skin tone, distinctive features, etc.)
- Any particularly prominent facial features or expressions
- Hairstyle (length, color, style, etc.)

Also, extract 10 key keywords and list them within <keywords> tags.
"""
    
    try:
        # Set up system prompt
        system_list = [
            {
                "text": system_prompt
            }
        ]
        
        # Set up user message (including image)
        message_list = [
            {
                "role": "user",
                "content": [
                    {
                        "image": {
                            "format": "png",
                            "source": {
                                "bytes": encoded_string
                            }
                        }
                    },
                    {
                        "text": user_prompt
                    }
                ]
            }
        ]
        
        # Set inference parameters
        inf_params = {"maxTokens": 1000, "temperature": 0.3}
        
        # Construct request body
        native_request = {
            "schemaVersion": "messages-v1",
            "messages": message_list,
            "system": system_list,
            "inferenceConfig": inf_params
        }
        
        # Make API request
        response = bedrock_runtime.invoke_model(
            modelId=model_id, 
            body=json.dumps(native_request)
        )
        
        # Process response
        model_response = json.loads(response["body"].read())
        result_text = model_response["output"]["message"]["content"][0]["text"]
        
        print("\nFace Analysis Results:")
        print(result_text)
        
        return result_text

    except ClientError as err:
        message = err.response['Error']['Message']
        logger.error("A client error occurred: %s", message)
        print(f"A client error occured: {message}")
        return None

## Running Face Analysis

In [ ]:
# Run face analysis
face_analysis = analyze_face(image_path)

## Step 2: Fortune Generation Function Implementation (Using Nova Model)

In [ ]:
def read_face(face_desc):
    """
    Function to generate fortune reading based on facial analysis results using Nova model
    """
    # Set model ID - Nova Lite model
    model_id = "us.amazon.nova-lite-v1:0"
    
    # Create Bedrock client
    bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')
    
    # Create prompt
    system_prompt = "You are a face reading expert. Your task is to generate a fortune reading based on the provided facial analysis information. Include only positive content in your reading, avoiding any negative interpretations."
    
    user_prompt = f"""
Based on the following facial analysis information, please provide a detailed fortune reading covering these areas:

1. Personality and general fortune based on face shape and overall impression
2. Wisdom and insight based on eye characteristics
3. Wealth and interpersonal relationships based on nose and mouth characteristics
4. Future prospects based on overall facial balance
5. Comprehensive face reading interpretation and advice

Facial Analysis Information:
{face_desc}
"""
    
    try:
        # Set up system prompt
        system_list = [
            {
                "text": system_prompt
            }
        ]
        
        # Set up user message
        message_list = [
            {
                "role": "user",
                "content": [
                    {
                        "text": user_prompt
                    }
                ]
            }
        ]
        
        # Set inference parameters
        inf_params = {"maxTokens": 2000, "temperature": 0.3}
        
        # Construct request body
        native_request = {
            "schemaVersion": "messages-v1",
            "messages": message_list,
            "system": system_list,
            "inferenceConfig": inf_params
        }
        
        # Make API request
        response = bedrock_runtime.invoke_model(
            modelId=model_id, 
            body=json.dumps(native_request)
        )
        
        # Process response
        model_response = json.loads(response["body"].read())
        result_text = model_response["output"]["message"]["content"][0]["text"]
        
        print("\nFortune Reading Results:")
        print(result_text)
        
        return result_text

    except ClientError as err:
        message = err.response['Error']['Message']
        logger.error("A client error occurred: %s", message)
        print(f"A client error occured: {message}")
        return None

## Running Fortune Generation

In [ ]:
# Generate fortune reading
fortune = read_face(face_analysis)

## Organizing and Displaying Results

In [ ]:
def display_results(face_analysis, fortune):
    """Function to organize and display analysis results and fortune readings"""
    
    # Generate HTML content
    html_content = f"""
    <div style="margin-bottom: 20px;">
        <h2 style="color: #2E86C1;">📊 Face Analysis Results</h2>
        <div style="background-color: #EBF5FB; padding: 15px; border-radius: 5px; white-space: pre-wrap;">
            {face_analysis}
        </div>
    </div>
    
    <div>
        <h2 style="color: #2E86C1;">🔮 Fortune Reading Results</h2>
        <div style="background-color: #EBF5FB; padding: 15px; border-radius: 5px; white-space: pre-wrap;">
            {fortune}
        </div>
    </div>
    """
    
    # Display HTML
    display(HTML(html_content))

## Displaying Results

In [ ]:
# Display results
display_results(face_analysis, fortune)

## Integrated Process Function

In [ ]:
def generate_face_fortune(image_path):
    """Integrated function to generate fortune reading from a face image"""
    # Display image
    load_and_display_image(image_path)
    
    # Step 1: Face Analysis
    print("Step 1: Analyzing facial features...")
    face_analysis = analyze_face(image_path)
    
    # Step 2: Fortune Generation
    print("\nStep 2: Generating fortune reading...")
    fortune = read_face(face_analysis)
    
    # Display results
    display_results(face_analysis, fortune)
    
    return face_analysis, fortune

# To test with another image, use the code below
# new_image_path = "sample/another_face.png"  # Another image path
# face_analysis, fortune = generate_face_fortune(new_image_path)